In [68]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import requests
import xmltodict
import math

In [44]:
## 법정동코드 불러오기
path = 'C:/Users/GCC/git'
bjd_cd_df = pd.read_csv(f'{path}/data/DL/법정동코드 전체자료.txt', encoding='cp949', sep='\t', dtype=str)
bjd_cd_df['시군구코드'] = bjd_cd_df['법정동코드'].str.slice(0, 5)
bjd_cd_df['시도코드'] = bjd_cd_df['법정동코드'].str.slice(0, 2)
bjd_cd_df['위상구분'] = '읍면동'
bjd_cd_df['위상구분'][bjd_cd_df['법정동코드'].apply(lambda x: x[5:]) == '00000'] = '시군구'
bjd_cd_df['위상구분'][bjd_cd_df['법정동코드'].apply(lambda x: x[2:5]) == '000'] = '시도'

In [85]:
authKey = 'G7GuVfz99A0umUDOAuZlRXMYlZmKxBd3hHsj8EiZ89SSD7r5qrn08q70LM96fyblxiM3eGxWsgkIjhF%2BAWTVBg%3D%3D'

## 건축물대장
def get_new_apt_quantity(sigungu_cd, bjd_cd, pageNo = 1, authKey = authKey):
    url = f"http://apis.data.go.kr/1613000/BldRgstService_v2/getBrTitleInfo?serviceKey={authKey}&sigunguCd={sigungu_cd}&bjdongCd={bjd_cd}&numOfRows=100&pageNo={pageNo}"
    response = requests.get(url)
    raw = xmltodict.parse(response.text)
    max_page = math.ceil(int(raw['response']['body']['totalCount'])/100)
    raw = raw['response']['body']['items']['item']
    return pd.DataFrame(raw), max_page

In [101]:
## 법정동별 건축물 표제부 조회
loca_cd_ls = bjd_cd_df[bjd_cd_df['위상구분'] == '읍면동'][['법정동코드', '시군구코드']].drop_duplicates().reset_index(drop=True)

_ls = []
for i in range(len(loca_cd_ls)):
    sigungu_cd = loca_cd_ls['시군구코드'][i]
    bjd_cd = loca_cd_ls['법정동코드'][i][5:]
    print(f"{i}//{len(loca_cd_ls)}") if i % 100 == 0 else None
    _df, max_page = get_new_apt_quantity(sigungu_cd, bjd_cd, 1, authKey)
    if max_page > 1:
        for i in range(2, max_page+1):
            _df = pd.concat([_df, get_new_apt_quantity(sigungu_cd, bjd_cd, i, authKey)[0]], axis=0)
    _ls.append(_df)

0//49349
100//49349
200//49349
300//49349
400//49349
500//49349
600//49349
700//49349
800//49349
900//49349
1000//49349
1100//49349
1200//49349
1300//49349
1400//49349
1500//49349


KeyError: 'response'

0


In [96]:
tt = pd.concat(_ls)

In [100]:
tt['mainPurpsCdNm'].unique()

array(['공동주택', '제1종근린생활시설', '단독주택', '제2종근린생활시설', '문화및집회시설', '교육연구시설',
       '종교시설', '노유자시설', '교육연구및복지시설', '교정및군사시설'], dtype=object)

1111010100


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [33]:
temp.keys()

dict_keys(['법정동코드', '시군구코드'])